In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
datasets = ['test']

In [3]:
outlet_churn_rate = pd.read_csv('data/outlet_churn_rate.csv')
outlet_churn_rate = outlet_churn_rate.groupby(['ownareaall_sqm', 'format_name']).agg({'churn_rate': 'mean'}).sort_values(by='churn_rate')
outlet_churn_rate.columns = ['churn_rate']

In [4]:
def check_sauces(text):
    sauces = ['соус', 'кетчуп']
    if any(sub in text for sub in sauces):
        return 1
    return 0 

def check_main_food(text):
    main_food = ['бургер', 'воппер', 'ролл', 'ангус', 'гранд чиз', 'цезарь', 'стейкхаус', 'чикен', 
                 'кинг букет', 'бекон кинг', 'острый инди', 'итальяно кинг', 'чеддер бекон', 'а4', 'инди пармезан', 
                 'острый чеддер', 'беконайзер', 'начос кинг', 'биг кинг', 'льзитер', 'конверт']
    if any(sub in text for sub in main_food):
        return 1
    return 0


def check_snacks(text):
    snacks = ['фри', 'крылышки', 'креветки', 'луковые', 'кольца', 'наггетс', 'стрипс', 'картофель', 
              'медальоны', 'карт.', 'кинг гоу', 'деревенский', 'начос бокс', 'попкорн', 'сухарики', 'чипсы']
    if any(sub in text for sub in snacks):
        return 1
    return 0

def check_cold_drinks(text):
    cold_drinks = ['мандарин', 'фрустайл ', 'пепси', 'миринда', 'дюшес', 'сок', 'кола', 'липтон', 
                   'эвервесс', 'байкал', 'лимонад', 'севен ап', 'вода', 'flash up', 'адреналин', ]
    if any(sub in text for sub in cold_drinks):
        return 1
    return 0

def check_hot_drinks(text):
    hot_drinks = ['кофе','эспрессо','латте','капучино','чай','какао']
    if any(sub in text for sub in hot_drinks):
        return 1
    return 0

def check_alco(text):
    alco = ['пиво', 'балтика']
    if any(sub in text for sub in alco):
        return 1
    return 0

def check_desert(text):
    desert = ['пирожок', 'рожок', 'улитка', 'сандэй', 'маффин', 'брауни', 'мороженое','айс ист', 'шейк',]
    if any(sub in text for sub in desert):
        return 1
    return 0

def outlet_id(row):
    try:
        return outlet_churn_rate.index.get_loc((row['ownareaall_sqm'], row['format_name']))
    except:
        return -1

def get_churn_rate(row):
    try:
        return outlet_churn_rate.iloc[row['outlet_id']]
    except:
        return 0.8

def ownareaall_category(ownareaall_sqm, low, medium):
        if ownareaall_sqm <= low:
            return 0
        if ownareaall_sqm <= medium:
            return 1
        return 2

In [5]:
for dataset in datasets:
    file_path = f'df_{dataset}.csv'
    df = pd.read_csv(file_path, parse_dates=[6])
    print('Начинаю обработку данных...')
    df['dish_name'] = df['dish_name'].str.lower()
    df['sauces'] = df['dish_name'].progress_apply(check_sauces)
    df['main_food'] = df['dish_name'].progress_apply(check_main_food)
    df['snacks'] = df['dish_name'].progress_apply(check_snacks)
    df['cold_drinks'] = df['dish_name'].progress_apply(check_cold_drinks)
    df['hot_drinks'] = df['dish_name'].progress_apply(check_hot_drinks)
    df['alco'] = df['dish_name'].progress_apply(check_alco)
    df['desert'] = df['dish_name'].progress_apply(check_desert)
    data_food_cat = df.groupby(['customer_id','startdatetime'])[['sauces', 'main_food', 'snacks', 'cold_drinks', 'hot_drinks', 'alco', 'desert']].sum()
    data_food_cat = data_food_cat.groupby('customer_id')[['sauces', 'main_food', 'snacks', 'cold_drinks', 'hot_drinks', 'alco', 'desert']].mean()
    print('data_food_cat done')

    data_pivot = pd.pivot_table(df, values=['buy_post', 'date_diff_post', 'format_name','ownareaall_sqm', 'revenue'], 
                       index=['customer_id','startdatetime'],
                       aggfunc={'buy_post': 'max', 'date_diff_post' : 'max','format_name': 'last', 'ownareaall_sqm': 'last', 'revenue': 'sum'})
    data_pivot = data_pivot.reset_index()

    data_pivot['outlet_id'] = data_pivot[['ownareaall_sqm', 'format_name']].progress_apply(outlet_id, axis=1)
    print('outlet_id done')

    data_pivot['churn_rate'] = data_pivot[['outlet_id']].progress_apply(get_churn_rate, axis=1)
    print('churn_rate done')

    cols_num = 3
    for i in range(1, cols_num + 1):
        data_pivot[f'{i}InvoiceDate'] = data_pivot.groupby('customer_id')['startdatetime'].shift(i)
    for i in range(1, cols_num + 1):
        data_pivot[f'{i}DayDiff'] = (data_pivot['startdatetime'] - data_pivot[f'{i}InvoiceDate']).dt.seconds
    data_day_diff = data_pivot.groupby('customer_id').agg({'1DayDiff': ['mean','std']}).reset_index()
    data_day_order_last = data_pivot.drop_duplicates(subset=['customer_id'],keep='last')
    data_day_order_last = data_day_order_last[['customer_id', '1DayDiff', '2DayDiff']]
    data_day_order_last = data_day_order_last.merge(data_day_diff, on='customer_id')
    data_day_order_last.columns = ['customer_id', 'day_diff_1', 'day_diff_2', 'day_diff_mean', 'day_diff_std']
    print('data_day_order_last done')

    data_area = data_pivot.groupby('customer_id').agg({'ownareaall_sqm': 'mean'}).reset_index()
    data_area.columns = ['customer_id','ownareaall_sqm_mean']
    print('data_area done')

    data_churn_rate = data_pivot.groupby('customer_id').agg({'churn_rate': ['mean','std'],
                                                         'outlet_id': lambda x: random.choice(pd.Series.mode(x))}).reset_index()
    data_churn_rate.columns = ['customer_id', 'churn_rate_mean', 'churn_rate_std', 'favourite_outlet_id']
    data_churn_rate['favourite_churn_rate'] = data_churn_rate['favourite_outlet_id'].apply(lambda x: outlet_churn_rate.iloc[x])
    print('data_churn_rate done')

    purchase_counts = data_pivot.groupby('customer_id').size()
    total_revenue_per_customer = data_pivot.groupby('customer_id')['revenue'].sum()
    average_revenue_per_customer = data_pivot.groupby('customer_id')['revenue'].mean()
    data_revenue = pd.DataFrame({
        'total_revenue': total_revenue_per_customer,
        'average_revenue': average_revenue_per_customer,
        'count': purchase_counts
    })
    data_revenue = data_revenue.reset_index()
    print('data_revenue done')
    data_target = data_pivot.groupby('customer_id').agg({'buy_post': 'last', 'date_diff_post': 'last'}).reset_index()
    print('data_target done')
    data_full = (data_revenue.merge(data_food_cat, on = 'customer_id')
             .merge(data_area, on = 'customer_id')
             .merge(data_churn_rate, on = 'customer_id')
             .merge(data_day_order_last, on = 'customer_id')
             .merge(data_target, on = 'customer_id'))
    data_full = data_full.dropna(subset=['day_diff_std'])
    print(data_full.shape)
    data_full.to_csv(f'data/data_{dataset}_full.csv')

Начинаю обработку данных...


  0%|          | 0/2431249 [00:00<?, ?it/s]

  0%|          | 0/2431249 [00:00<?, ?it/s]

  0%|          | 0/2431249 [00:00<?, ?it/s]

  0%|          | 0/2431249 [00:00<?, ?it/s]

...

outlet_id done


  0%|          | 0/577178 [00:00<?, ?it/s]

churn_rate done


C:\Users\sergey_qt\AppData\Local\Temp\ipykernel_12620\3752089803.py:36: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  data_day_order_last = data_day_order_last.merge(data_day_diff, on='customer_id')
C:\Users\sergey_qt\AppData\Local\Temp\ipykernel_12620\3752089803.py:36: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  data_day_order_last = data_day_order_last.merge(data_day_diff, on='customer_id')


data_day_order_last done
data_area done
data_churn_rate done
data_revenue done
data_target done
(100000, 22)
